<a href="https://colab.research.google.com/github/hmghaly/word_align/blob/main/walign2022_NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')
cwd='/content/drive/MyDrive/tr_ar' #directory where we keep the data
if not os.path.exists(cwd): os.makedirs(cwd)  #if the directory doesn't exist, create it
os.chdir(cwd) #change current work directory to the directory where we keep the data
!pwd

Mounted at /content/drive
/content/drive/MyDrive/tr_ar


In [2]:
#Clone the github codebase
!rm -r code_utils #first delete the code_utils folder, then clone from the codebase (very crude till we figure out using github from colab)
!git clone https://github.com/hmghaly/word_align.git code_utils

Cloning into 'code_utils'...
remote: Enumerating objects: 878, done.
remote: Counting objects: 100% (692/692), done.
remote: Compressing objects: 100% (659/659), done.
remote: Total 878 (delta 118), reused 534 (delta 24), pack-reused 186
Receiving objects: 100% (878/878), 26.36 MiB | 13.61 MiB/s, done.
Resolving deltas: 100% (219/219), done.


In [3]:
from code_utils.general import *
from code_utils.align_utils import *
from code_utils.arabic_lib import *

from itertools import groupby
from collections import Counter
ar_counter_dict=load_counts("ar_count.txt", tmp_count_dict={})

def cur_ar_tok_fn(text):
  cur_toks=tok(text)
  return tok_ar(cur_toks,ar_counter_dict)
print("loaded necessary functions")

loaded necessary functions


#Loading and indexing sentences

In [4]:
ar_corpus_fname="UNv1.0.ar-en-clean2.ar"
en_corpus_fname="UNv1.0.ar-en.en"

ar_fopen=open(ar_corpus_fname)
en_fopen=open(en_corpus_fname)

src_sents,trg_sents=[],[]
for i,pair in enumerate(zip(en_fopen,ar_fopen)):
  if i>=100000: break
  if i%20000==0: print(i)
  en_line,ar_line=pair
  en_line,ar_line=en_line.strip("\n\r\t "),ar_line.strip("\n\r\t ")
  src_sents.append(en_line)
  trg_sents.append(ar_line)

ar_fopen.close()
en_fopen.close()
print(len(src_sents),len(src_sents))
print("Now adding another bitext in a Google sheet")
import pandas as pd
xls_path='https://docs.google.com/spreadsheets/d/e/2PACX-1vSqipSUAaMWJrizvZtg1WIpds2pIiXtcZC6wjOLeVYie9p5O0lqiPnVdd8ambGKH8B0qJ4kmlVeEs6f/pub?output=xlsx'
cur_xls = pd.ExcelFile(xls_path)
cur_sheet_name="2"
cur_sheet_obj=pd.read_excel(cur_xls, cur_sheet_name,keep_default_na=False)
sent_pair_list=[]
for i,row in enumerate(cur_sheet_obj.iterrows()):
  row_dict=row[1].to_dict()
  en0,ar0=row_dict["en"],row_dict["ar"]
  ar0=clean_ar(ar0)
  sent_pair_list.append((en0,ar0))
  src_sents.append(en0)
  trg_sents.append(ar0)

#Now indexing
print("Now indexing all sentences")
src_indexing_obj=indexing(src_sents,tok,stop_words=["the","of"],ignore_punc=True)
print("finihsed indexing source sentences")
trg_indexing_obj=indexing(trg_sents,cur_ar_tok_fn,ignore_ar_pre_suf=False,ignore_punc=True)
print("finihsed indexing target sentences")

0
20000
40000
60000
80000
100000 100000
Now adding another bitext in a Google sheet
Now indexing all sentences
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
finihsed indexing source sentences
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
finihsed indexing target sentences


#Starting Alignmenet

In [5]:
import math
src_list=src_indexing_obj.all_tok_sentences
trg_list=trg_indexing_obj.all_tok_sentences
src_inv_dict=src_indexing_obj.inv_index
trg_inv_dict=trg_indexing_obj.inv_index

def overlaps(span1,span2):
  span1_i0,span1_i1=span1
  span2_i0,span2_i1=span2
  if span1_i0>=span2_i0 and span1_i0<=span2_i1: return True
  if span1_i1>=span2_i0 and span1_i1<=span2_i1: return True
  if span2_i0>=span1_i0 and span2_i0<=span1_i1: return True
  if span2_i1>=span1_i0 and span2_i1<=span1_i1: return True
  return False

def get_inner_dist(span1,span2):
  combined_xs=[]
  combined_xs.extend(span1)
  combined_xs.extend(span2)  
  combined_xs.sort()
  return combined_xs[2]-combined_xs[1]

def get_rec_children(span0,span_child_dict0,span_el_dict0,el_list0=[]):
  cur_children0=span_child_dict0.get(span0,[])
  for ch0 in cur_children0:
    el_list0.append(span_el_dict0[ch0])
    el_list0=get_rec_children(ch0,span_child_dict0,span_el_dict0,el_list0)
  return el_list0



def get_min_max(range1,range2):
  range1_x0,range1_x1=range1
  range2_x0,range2_x1=range2
  min_x=min(range1_x0,range1_x1,range2_x0,range2_x1)
  max_x=max(range1_x0,range1_x1,range2_x0,range2_x1)
  return (min_x,max_x)

def get_dist_factor(el1,el2): #[(5,6),(9,9)] - [(7,7),(12,13)] #get inner distance between two elements, 1 if adjacent
  el1_xrange,el1_yrange=el1
  el2_xrange,el2_yrange=el2
  combined_xs=[]
  combined_xs.extend(el1_xrange)
  combined_xs.extend(el2_xrange)
  combined_ys=[]
  combined_ys.extend(el1_yrange)
  combined_ys.extend(el2_yrange)
  combined_xs.sort()
  combined_ys.sort()
  inner_xdist=combined_xs[2]-combined_xs[1]
  inner_ydist=combined_ys[2]-combined_ys[1]
  return (inner_xdist+inner_ydist)/2

def get_area_factor(x_range1,y_range1):
  x_dist=x_range1[1]-x_range1[0]+1
  y_dist=y_range1[1]-y_range1[0]+1
  cur_factor=(x_dist+y_dist)/2
  return cur_factor #math.log(cur_factor)


def get_tok_loc(tok_list):
  tok_locs_list=[]
  for i_,cur_tok in enumerate(tok_list):
    tok_locs_list.append(((cur_tok,),(i_,i_)))
    if i_<len(tok_list)-1: tok_locs_list.append(((cur_tok,tok_list[i_+1]), (i_,i_+1)))
  tok_locs_list.sort(key=lambda x:x[0])
  grouped=[(key,[v[1] for v in list(group)]) for key,group in groupby(tok_locs_list,lambda x:x[0])]
  return grouped


test_i=-len(sent_pair_list)
n_sents=len(sent_pair_list)-2
test_i=101
n_sents=1
sent_counter=0
all_phrase_pairs=[]
for src0,trg0 in zip(src_list[test_i:test_i+n_sents],trg_list[test_i:test_i+n_sents]):
  print(sent_counter)
  print(src0)
  print(trg0)
  span_el_dict={} #which element corresponds to this span
  span_wt_dict={} #wight of each span
  span_child_dict={} #which subspans this span consists of
  src0=['<s>']+src0+['</s>']
  trg0=['<s>']+trg0+['</s>']
  src_grouped=get_tok_loc(src0) #group all the locations for a list of consecutive tokens (src then trg)
  trg_grouped=get_tok_loc(trg0)
  src_side,trg_side=[],[]

  for s_toks,s_locs_grp in src_grouped: #retrive the indexes for each token sequence, map them to the locations
    src_indexes0= rtrv_idx(s_toks,src_inv_dict)
    src_side.append((s_toks,s_locs_grp,src_indexes0))
  for t_toks,t_locs_grp in trg_grouped:
    trg_indexes0= rtrv_idx(t_toks,trg_inv_dict)
    trg_side.append((t_toks,t_locs_grp,trg_indexes0))

  #now starting the match the indexes of both src and trg token locations
  matching_list=[]
  all_elements=[] #to create elements (pairs of src-trg spans)
  tmp_ratio_dict={}
  for s_toks,s_locs_grp,src_indexes0 in src_side:
    s_toks_str=" ".join(s_toks)
    #continue
    #print(s_toks,s_locs_grp,len(src_indexes0))
    for t_toks,t_locs_grp,trg_indexes0 in trg_side:
      cur_ratio=0
      t_toks_str=" ".join(t_toks)
      if s_toks==t_toks: cur_ratio=1
      else: 
        #cur_ratio=idx_match(src_indexes0,trg_indexes0)
        if len(src_indexes0)==0 or len(trg_indexes0)==0: continue #cur_ratio= 0
        intersection=len(list(set(src_indexes0).intersection(set(trg_indexes0))))
        if intersection<10: continue
        union=len(src_indexes0)+len(trg_indexes0)-intersection
        if union==0: continue #cur_ratio= 0
        cur_ratio=float(intersection)/union
      if cur_ratio==0: continue
      for sl0 in s_locs_grp:
        for tl0 in t_locs_grp:
          cur_el=(sl0,tl0) #an element is a tuple containing the span of src indexes and span of trg indexes
          tmp_ratio_dict[cur_el]=cur_ratio
          #all_elements.append((cur_el,s_toks_str,t_toks_str,cur_ratio))
          all_elements.append((cur_el,cur_ratio))
          found_wt=span_wt_dict.get(sl0,0)
          if found_wt<cur_ratio:
            span_wt_dict[sl0]=cur_ratio
            span_el_dict[sl0]=cur_el
            span_child_dict[sl0]=[]



      #matching_list.append((s_toks,t_toks,cur_ratio))
  #and now let's group all the elements by the starting location of the src tokenized sentence
  all_elements.sort(key=lambda x:(x[0][0][0],-x[-1]))
  all_elements_grouped=[(key,list(group)) for key,group in groupby(all_elements,lambda x:x[0][0][0])]
  el_start_dict={}
  for k0,el_grp in all_elements_grouped:
    el_start_dict[k0]=el_grp[:10] #for each starting src index , we identify the top 10 elements that start with it
  
  for inc0 in range(len(src0)): #increments to cover slices of src location indexes
    for i0 in range(len(src0)):
      i1=i0+inc0
      if i1>len(src0)-1: continue
      if inc0>15: continue
      #print(i0,i1)
      first_items=el_start_dict.get(i0,[])
      next_items=[]
      for tmp_i in range(i0+1,i1+1):
        next_items+=el_start_dict.get(tmp_i,[])
        #print(tmp_i)
      tmp_final_list=list(first_items)
      
      for fi in first_items:
        fi_element,fi_cur_ratio=fi
        fi_src_range,fi_trg_range=fi_element
        fi_src_i0,fi_src_i1=fi_src_range
        fi_trg_i0,fi_trg_i1=fi_trg_range

        for ni in next_items:
          ni_element,ni_cur_ratio=ni
          ni_src_range,ni_trg_range=ni_element
          ni_src_i0,ni_src_i1=ni_src_range
          ni_trg_i0,ni_trg_i1=ni_trg_range   
           #print("<<<<<<<<<<")
          # up=False
          # if ni_trg_i1<fi_trg_i0: up=True
          trg_spans_overlap=overlaps(fi_trg_range,ni_trg_range)
          trg_span_inner_dist=get_inner_dist(fi_trg_range,ni_trg_range)

          if ni_src_i0<=fi_src_i1: continue #skip if src spans overlap
          if trg_spans_overlap: continue #skip if target spans overlap
          if trg_span_inner_dist>10: continue #skip if there is much distance between target spans

          # print("fi_src_range",fi_src_range,"fi_trg_range",fi_trg_range)
          # print("ni_src_range",ni_src_range,"ni_trg_range",ni_trg_range) 
          #if ni_trg_i0<=fi_trg_i1: print("<<<<<<<<<<")
          #if up: print(">>>> UP")
          #if trg_spans_overlap: print(">>>> OVERLAPS")
          #print("trg_span_inner_dist",trg_span_inner_dist)
          # print("-----------") 
          # continue     
          # new_src_range=fi_src_range[0],ni_src_range[-1]
          # new_trg_range=fi_src_range[0],ni_src_range[-1]
          new_src_range=get_min_max(fi_src_range,ni_src_range)
          new_trg_range=get_min_max(fi_trg_range,ni_trg_range)
          combined_ratio=fi_cur_ratio+ni_cur_ratio
          cur_area_factor=get_area_factor(new_src_range,new_trg_range)

          # print("combined_ratio",combined_ratio)
          # print("-----------") 
          # continue   
          #cur_area_factor=get_dist_factor(fi_cur_ranges,ni_cur_ranges)
          if cur_area_factor<1: continue
          #adj_ratio=combined_ratio/cur_area_factor
          #adj_ratio=combined_ratio
          adj_ratio=combined_ratio-trg_span_inner_dist*0.05-cur_area_factor*0.005
          cur_el=(new_src_range,new_trg_range)
          found_span_wt=span_wt_dict.get(new_src_range,0)
          if adj_ratio>found_span_wt:
            span_wt_dict[new_src_range]=adj_ratio
            span_el_dict[new_src_range]=cur_el
            span_child_dict[new_src_range]=[fi_src_range,ni_src_range]
          found_ratio=tmp_ratio_dict.get(cur_el,0)
          if adj_ratio>found_ratio:
            tmp_final_list.append((cur_el,adj_ratio))
          #print(fi_cur_ranges,ni_cur_ranges,"new_src_range",new_src_range,"new_trg_range",new_trg_range,"combined_ratio", combined_ratio,"cur_area_factor",cur_area_factor)
      
      tmp_final_list.sort(key=lambda x:-x[-1])
      used_els=[]
      final_items=[]
      for tp in tmp_final_list:
        el0,ratio0=tp
        if el0 in used_els: continue
        used_els.append(el0)
        final_items.append(tp)
        src_range0,trg_range0=tp[0]        
        #print(tp)
        cur_src_str=" ".join([src0[i_] for i_ in range(src_range0[0],src_range0[1]+1)])
        cur_trg_str=" ".join([trg0[i_] for i_ in range(trg_range0[0],trg_range0[1]+1)])
        # print(">>>", round(ratio0,4),src_range0,trg_range0)
        # print(cur_src_str)
        # print(cur_trg_str)
        #print("^^")
        #print([src0[i_] for i_ in range(src_range0[0],src_range0[1]+1)])
        #print([trg0[i_] for i_ in range(trg_range0[0],trg_range0[1]+1)])
        #if len(final_items)>10: break
      #print("---")
      final_items.sort(key=lambda x:-x[-1])
      el_start_dict[i0]=final_items[:50]

  list_spans_wts=[]
  for a,b in span_wt_dict.items():
    list_spans_wts.append((a,b))
  
  list_spans_wts.sort(key=lambda x:-x[-1])
  #get_rec_children(span0,span_child_dict0,span_el_dict0,el_list0=[])
  print(list_spans_wts[0])
  all_elements=get_rec_children(list_spans_wts[0][0],span_child_dict,span_el_dict,el_list0=[])
  for ch_el in all_elements:
    print(ch_el)
    src_span0,trg_span0=ch_el
    cur_src_str=" ".join([src0[i_] for i_ in range(src_span0[0],src_span0[1]+1)])
    cur_trg_str=" ".join([trg0[i_] for i_ in range(trg_span0[0],trg_span0[1]+1)])  
    cur_pair=(cur_src_str,cur_trg_str)
    print(cur_src_str)   
    print(cur_trg_str) 
    print("--") 
    all_phrase_pairs.append((cur_pair,sent_counter))
  sent_counter+=1

for a in all_phrase_pairs:
  print(a)
      # for a,b in list_spans[:2]:
      #   #print()
      #   cur_children=span_child_dict[a]
      #   cur_el=span_el_dict[a]
      #   print(a,b, ">>> children:", cur_children)
      #   #print("element:",cur_el)
      #   for ch in cur_children:
      #     ch_el=span_el_dict[ch]
      #     ch_wt=span_wt_dict[ch]
      #     src_span0,trg_span0=ch_el
      #     cur_src_str=" ".join([src0[i_] for i_ in range(src_span0[0],src_span0[1]+1)])
      #     cur_trg_str=" ".join([trg0[i_] for i_ in range(trg_span0[0],trg_span0[1]+1)]) 
      #     print(ch_el,ch_wt)
      #     print(cur_src_str)   
      #     print(cur_trg_str) 
      #     print("--")     

      #   print("========")
      # for a,b in span_child_dict.items():
      #   print(a,b)
      # for a,b in span_el_dict.items():
      #   print(a,b)
      #print("first_items", len(first_items),first_items[:2])
      #print("next_items", len(next_items),next_items[:2])

0
['21', 'australia', 'notes', 'that', 'concept', 'ceded', 'jurisdiction', 'advanced', 'in', 'paragraph', '456', 'working', 'group', 'report', 'offers', 'only', 'a', 'partial', 'solution', 'to', 'resolving', 'possible', 'conflicts', 'between', 'a', 'state', 'jurisdiction', 'under', 'an', 'existing', 'treaty', 'and', 'its', 'acceptance', 'a', 'court', 'jurisdiction']
['21', 'وـ', 'تلاحظ', 'استراليا', 'أن', 'مفهوم', 'نقل', 'اختصاص', 'وارد', 'في', 'فقرة', '456', 'من', 'تقرير', 'فريق', 'عامل', 'لا', 'يعطي', 'سوى', 'حل', 'جزئي', 'لـ', 'تسوية', 'منازعات', 'ممكنة', 'بين', 'اختصاص', 'دولة', 'بموجب', 'معاهدة', 'قائمة', 'وـ', 'قبول', 'ـها', 'اختصاص', 'محكمة', 'مرتآة']
((1, 38), 11.683997840015618)
((1, 14), (1, 16))
21 australia notes that concept ceded jurisdiction advanced in paragraph 456 working group report
21 وـ تلاحظ استراليا أن مفهوم نقل اختصاص وارد في فقرة 456 من تقرير فريق عامل
--
((1, 5), (1, 6))
21 australia notes that concept
21 وـ تلاحظ استراليا أن مفهوم
--
((1, 3), (1, 4))
21 aust

In [86]:
fopen=open("test-out-align.txt","w")
for a in all_phrase_pairs:
  line=" ".join([str(v) for v in a])+"\n"
  fopen.write(line)
fopen.close()
  #print(a)

In [90]:
from collections import Counter
just_pairs=[]
for a in all_phrase_pairs:
  pair,sent_i=a
  src0,trg0=pair
  just_pairs.append(pair)
just_pairs.sort()
grouped=[(key,[v[1] for v in list(group)]) for key,group in groupby(just_pairs,lambda x:x[0])]
grouped.sort(key=lambda x:len(x[1]))
for k0,grp0 in grouped:
  print(k0)
  #print(grp0)
  print(Counter(grp0))
  print("----")

Streaming output truncated to the last 5000 lines.
Counter({'لـ قياس قدرة': 1})
----
mechanism
Counter({'آلية': 1})
----
mechanisms were
Counter({'كانت آليات': 1})
----
meeting
Counter({'اجتماع': 1})
----
meetings
Counter({'اجتماعات': 1})
----
member delegation concerned
Counter({'أعضاء وفد معني': 1})
----
member states at
Counter({'دول أعضاء في': 1})
----
member states at current meeting
Counter({'دول أعضاء في اجتماع حالي': 1})
----
member states but
Counter({'دول أعضاء ولكن': 1})
----
member states but was nevertheless
Counter({'دول أعضاء ولكن ـها مع ذلك': 1})
----
member states but was nevertheless essential to ensuring smooth operation organization </s>
Counter({'دول أعضاء ولكن ـها مع ذلك ضرورية لـ ضمان سلاسة عمل منظمة </s>': 1})
----
member states capacity
Counter({'قدرة دول أعضاء': 1})
----
member states capacity to
Counter({'إلى قدرة دول أعضاء': 1})
----
member states capacity to pay and on
Counter({'إلى قدرة دول أعضاء على دفع وـ': 1})
----
member states capacity to pay and on m

In [ ]:
def get_dist_factor(el1,el2): #[(5,6),(9,9)] - [(7,7),(12,13)] #get inner distance between two elements, 1 if adjacent
  el1_xrange,el1_yrange=el1
  el2_xrange,el2_yrange=el2
  combined_xs=[]
  combined_xs.extend(el1_xrange)
  combined_xs.extend(el2_xrange)
  combined_ys=[]
  combined_ys.extend(el1_yrange)
  combined_ys.extend(el2_yrange)
  combined_xs.sort()
  combined_ys.sort()
  inner_xdist=combined_xs[2]-combined_xs[1]
  inner_ydist=combined_ys[2]-combined_ys[1]
  return (inner_xdist+inner_ydist)/2
test_el1=[(5,6),(9,9)]
test_el2=[(6,6),(10,11)]
print(get_dist_factor(test_el1,test_el2))

0.5


In [17]:
def overlaps(span1,span2):
  span1_i0,span1_i1=span1
  span2_i0,span2_i1=span2
  if span1_i0>=span2_i0 and span1_i0<=span2_i1: return True
  if span1_i1>=span2_i0 and span1_i1<=span2_i1: return True
  if span2_i0>=span1_i0 and span2_i0<=span1_i1: return True
  if span2_i1>=span1_i0 and span2_i1<=span1_i1: return True
  return False
sp1=(2,3)
sp2=(4,4)
print(overlaps(sp1,sp2))

False


In [22]:
def get_inner_dist(span1,span2):
  combined_xs=[]
  combined_xs.extend(span1)
  combined_xs.extend(span2)  
  combined_xs.sort()
  return combined_xs[2]-combined_xs[1]
sp1=(22,30)
sp2=(14,16)
print(get_inner_dist(sp1,sp2))

6


In [30]:
for ix in range(1,10):
  print(ix,math.log(ix))

1 0.0
2 0.6931471805599453
3 1.0986122886681098
4 1.3862943611198906
5 1.6094379124341003
6 1.791759469228055
7 1.9459101490553132
8 2.0794415416798357
9 2.1972245773362196


In [47]:
import pandas as pd
xls_path='https://docs.google.com/spreadsheets/d/e/2PACX-1vSqipSUAaMWJrizvZtg1WIpds2pIiXtcZC6wjOLeVYie9p5O0lqiPnVdd8ambGKH8B0qJ4kmlVeEs6f/pub?output=xlsx'
cur_xls = pd.ExcelFile(xls_path)
cur_sheet_name="2"
cur_sheet_obj=pd.read_excel(cur_xls, cur_sheet_name,keep_default_na=False)
sent_pair_list=[]
for i,row in enumerate(cur_sheet_obj.iterrows()):
  row_dict=row[1].to_dict()
  sent_pair_list.append((row_dict["en"],row_dict["ar"]))

for a in sent_pair_list:
  print(a)
# print(cur_sheet_obj.keys())

('A/C.5/76/SR.2', 'A/C.5/76/SR.2')
('A/C.5/76/SR.2', 'A/C.5/76/SR.2')
('United Nations', 'الأمم المتحدة')
('A/C.5/76/SR.2', 'A/C.5/76/SR.2')
('General Assembly', 'الجمعية العامة')
('Seventy-sixth session', 'الدورة السادسة والسبعون')
('Official Records', 'وثيقة رسمية')
('Distr.: General', 'Distr.: General')
('24 January 2022', '24 January 2022')
('Original: English', 'Arabic Original: English')
('21-14241', '21-14241')
('This record is subject to correction.', 'هذا المحضر قابل للتصويب.')
('Corrections should be sent as soon as possible, under the signature of a member of the delegation concerned, to the Chief of the Documents Management Section (dms@un.org), and incorporated in a copy of the record.', 'وينبغي إدراج التصويبات في نسخة من المحضر مذيلة بتوقيع أحد أعضاء الوفد المعني وإرسالها في أقرب وقت ممكن إلى: Chief of the Documents Management Section (dms@un.org).')
('Corrected records will be reissued electronically on the Official Document System of the United Nations (http://documents